In [10]:
import pandas as pd
import itertools

Option 1 is to pick a column as a list and do the trick with standard python list tools

In [16]:
#df=pd.read_excel('./testdata/testdata.xlsx')
#a dummy version to play with very small sets is
data = [['tom', "1"], ['jason', "13"], ['juli', 3], ['juli', 3], ['louise', 3] , 
        ['peter', 4],['mark', 10], ['nick', 2], ['andrea', 14], ['james', 0], ['john', ''], ['linn', 'nan'],['simon', "error"]  ] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['name', 'customer_id']) 
 
field_to_check='customer_id'
df[field_to_check]=pd.to_numeric(df[field_to_check], errors='coerce')
#the following method is if they are all strings
#it is faster
#df[field_to_check]= df[df[field_to_check].apply(lambda x: x.isnumeric())]

df.fillna(0,inplace=True)
df[field_to_check]=df[field_to_check].astype(int)
df

,name,customer_id
0,tom,1
1,jason,13
2,juli,3
3,juli,3
4,louise,3
5,peter,4
6,mark,10
7,nick,2
8,andrea,14
9,james,0


In [18]:
duplicated_rows = df[df.duplicated([field_to_check],keep='first')]
print("Duplicates",len(duplicated_rows))
duplicated_rows

Duplicates 5


,name,customer_id
3,juli,3
4,louise,3
10,john,0
11,linn,0
12,simon,0


In [19]:
df.drop_duplicates(subset=[field_to_check],keep='first', inplace=True)

In [20]:
list_to_check= df[field_to_check].tolist()
# here the key command is "set" which picks only unique values.
# note that this means that we dont check for duplicates or
# non numeric, blanks etc, this should be done beforehand

list_to_check=sorted(set(list_to_check))
#list_to_check.sort()
gap_list=[]
n= 0
for el in list_to_check:
    if n==0:
        n= el-1    
    if el-n > 1:
        gap_list.extend(range(n+1,el))
    n=el
gap_list

[5, 6, 7, 8, 9, 11, 12]

this small esoteric routine converts it to pair of ranges, which can be useful if the gaps are clustered

In [11]:
def to_ranges(iterable):
    iterable = sorted(set(iterable))
    for key, group in itertools.groupby(enumerate(iterable), lambda t: t[1] - t[0]):
        group = list(group)
        yield group[0][1], group[-1][1]
list(to_ranges(gap_list))

[(5, 9), (11, 12)]

Option 2 is to do everything with panda data frames

In [12]:
df.sort_values(by=[field_to_check], inplace=True)
gap_list=[]
n= 0
for index, row in df.iterrows():
    if n==0:
        n= row[field_to_check]-1    
    if el-n > 1:
        gap_list.extend(range(n+1,row[field_to_check]))
    n=row[field_to_check]
gap_list

Duplicates 5


[5, 6, 7, 8, 9, 11, 12]

In [21]:
df2 = pd.DataFrame({'id':gap_list})
df2.to_excel("Sequence Gaps.xlsx")
print(len(gap_list))

7
